In [2]:
!pip install gensim
!pip install nltk

You should consider upgrading via the 'c:\users\aniru\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\aniru\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [3]:
import numpy as np
import pandas as pd
import os
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from gensim import corpora, models
from pprint import pprint
from tqdm import tqdm

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aniru\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
os.chdir('..')

In [5]:
!pwd

C:\Users\aniru\Desktop\Amrita\Sem 5\Big Data and Database Management\Project\Kisan-Query-Analysis


## Functions

In [45]:
def lemmatize_stemming(text):
    stemmer = PorterStemmer()
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in tqdm(enumerate(ldamodel[corpus])):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

## Load required variables

In [44]:
dictionary_qt = pickle.load(open("Data\\models\\dictionary", "rb"))
tfidf_qt = pickle.load(open("Data\\models\\tfidf", "rb"))
lda_model_qt_tfidf = pickle.load(open("Data\\models\\lda_model_tfidf", "rb"))
corpus_qt_tfidf = pickle.load(open("Data\\models\\corpus_tfidf", "rb"))
corp = pickle.load(open("Data\\models\\corp_tfidf", "rb"))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [54]:
df_dominant_topic = pd.read_csv('Data\\Dominant_Topic_for_Queries.csv')
data = pd.read_csv('Data\\query_agg.csv')

c:\users\aniru\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [55]:
df_dominant_topic = df_dominant_topic[df_dominant_topic['Text'].notna()]

In [57]:
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
7,7,0.0,0.6363,"weather, inform, meali, husbandari, take, addr...",asking about weather forcasting?
13,13,3.0,0.5801,"report, fertil, dose, activ, cucumb, potato, a...",new varieties of paddy?
48,48,6.0,0.6208,"market, ask, pest, rate, suck, cotton, cultiv,...",# INFORMATION REGARDING TO CONTROL OF WEEDS ...
72,72,6.0,0.5746,"market, ask, pest, rate, suck, cotton, cultiv,...",WHETHER TO NEED IRRIGATE WHEAT 15 DAYS BEFORE ...
80,80,6.0,0.7317,"market, ask, pest, rate, suck, cotton, cultiv,...",control of stem rot in barseam?
89,89,3.0,0.3331,"report, fertil, dose, activ, cucumb, potato, a...",INFORMATION REGARDING FOR THE CONTROL OF APHID...
105,105,3.0,0.6571,"report, fertil, dose, activ, cucumb, potato, a...",INFORMATION REGARDING FOR THE CONTROL OF HAIRY...
135,135,1.0,0.4174,"tell, varieti, nutrient, point, manag, ask, go...",information regarding weather condition?
160,160,2.0,0.6205,"forecast, weather, ginger, vaijapur, aurangaba...",INFORMATION REGARDING LATEST VARIETIES OF RICE...
189,189,7.0,0.4119,"district, blight, block, murshidabad, bankura,...",what is the contact number of pau?


In [58]:
data.head(10)

,Season,Sector,Category,Crop,QueryType,QueryText,KccAns,StateName,DistrictName,BlockName,CreatedOn
0,NaN,AGRICULTURE,Cereals,Wheat,0,INFORMATION REGARDING TO THE CONTROL OF YELLOW...,SPRAY OF TILT(PROPICONAZOLE) @ 200 ML IN THE 2...,PUNJAB,AMRITSAR,AJNALA-1,2013-04-01T06:51:45
1,NaN,HORTICULTURE,Vegetables,Chillies,0,INFORMATION REGARDING TO CONTROL OF PHALARIS M...,THE PHALARIS MINOR CAN BE CONTROLLED IN CHILLI...,PUNJAB,AMRITSAR,AJNALA-1,2013-04-01T07:29:19
2,RABI,HORTICULTURE,Vegetables,1282,76,control of worm in bhindi?,NaN,PUNJAB,AMRITSAR,AJNALA-1,2013-04-01T07:35:38
3,KHARIF,AGRICULTURE,Cereals,Wheat,3,INFORMATION REGARDING TO THE CONTROL OF STEM B...,SPRAY OF EKALUX[QUINALPHOS] 25EC @ 800 ML IN T...,PUNJAB,AMRITSAR,AJNALA-1,2013-04-01T07:48:52
4,JAYAD,AGRICULTURE,Pulses,1064,19,seed rate of moong?,15kg per acre for 668and 12kg per acre for 832.,PUNJAB,AMRITSAR,RAYYA-6,2013-04-01T09:09:12
5,JAYAD,AGRICULTURE,Cereals,1075,3,INFORMATION REGARDING FOR THE CONTROL OF MAIZE...,FOR THE CONTROL OF MAIZE BORER ATTACK ON SUMME...,PUNJAB,AMRITSAR,AJNALA-1,2013-04-01T10:28:45
6,JAYAD,AGRICULTURE,Cereals,1137,5,INFORMATION REGARDING FOR THE SUITABLE VARIETI...,"SUITABLE VARIETIES OF PADDY(JHONA)-PR-121,PR-1...",PUNJAB,AMRITSAR,RAYYA-6,2013-04-01T10:50:33
7,KHARIF,AGRICULTURE,Cereals,8469,0,asking about weather forcasting?,it may be rainfall on 2rd april,PUNJAB,AMRITSAR,AJNALA-1,2013-04-01T11:07:47
8,KHARIF,AGRICULTURE,Cereals,Wheat,29,HOW TO CONTROL YELLOW RUST IN WHEAT?,SPRAY OF TILT(PROPICONAZOLE) @ 200 ML IN THE 2...,PUNJAB,AMRITSAR,AJNALA-1,2013-04-01T11:08:47
9,NaN,HORTICULTURE,Vegetables,1282,78,varieties of bhindi?,"punjab-8,7 padmani",PUNJAB,AMRITSAR,VERKA-5,2013-04-01T11:44:06


## Pipeline:

In [59]:
def pipeline(tt):
    tt = pd.Series(tt)
    processed = tt.map(preprocess)
#     dictionary = gensim.corpora.Dictionary(processed)
    bow_corpus = [dictionary_qt.doc2bow(doc) for doc in processed]
#     tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf_qt[bow_corpus]
#     print("Processed: {} \n BoW: {} \n Corpus TF-IDF: {}".format(processed, bow_corpus, corpus_tfidf[0]))
#     ctr = True
#     for index, score in sorted(lda_model_qt_tfidf[bow_corpus[0]], key=lambda tup: -1*tup[1]):
#         print("\nScore: {}\t \nTopic: {}".format(score, lda_model_qt_tfidf.print_topic(index, 8)))
    
    tops = lda_model_qt_tfidf.get_document_topics(corpus_tfidf[0])
    max1 = (0,0)
    for i in tops:
        if(i[1] > max1[1]):
            max1 = i
    aa1 = df_dominant_topic[df_dominant_topic["Text"].notna()]
    pos_doc = aa1[aa1["Dominant_Topic"] == max1[0]][:10]
    a1 = pos_doc["Text"].tolist()
#     fin_ans = dd[dd['QueryText'] == a1]['KccAns']
    
    fin_ans = []
    for i in data.values:
        if(i[5] in a1):
            fin_ans.append(i[6])
    return fin_ans[:10]

In [63]:
aa = "weather"
fin = pipeline(aa)
fin

['it may be rainfall on 2rd april',
 'Mostly dry. Warm (max 37°C on Tue afternoon, min 24°C on Sun night). Wind will be generally light.',
 'spray of 60 ml alfamethrin with 100 ltr water',
 'Some drizzle, heaviest during Thu morning. Warm (max 35°C on Wed afternoon, min 25°C on Tue night). Wind will be generally light.',
 '6 MARLE /8KG SEED',
 'Mostly dry. Warm (max 37°C on Tue afternoon, min 26°C on Sun night). Wind will be generally light.',
 'SOME CHANCES OF RAIN IN NEXT 24 HOURS',
 'SPRAY RIPCORD (10 EC) @ 200 ML IN 150 LITRE WATER/ACRE.',
 'SOME CHANCES OF RAIN IN NEXT 24 HOURS',
 'SOME CLOUDS AND NO CHANCES OF RAINFALL .']

In [64]:
aa = "pesticides"
fin = pipeline(aa)
fin

['SUITABLE VARIETIES OF PADDY(JHONA)-PR-121,PR-122,PR-118,PR-114,PR-111,PR-116,PR-115,PR-113,PUSA-44,HKR-47,PUSA PUNJAB BASMATI-1509,PUNJAB BASMATI-3.',
 'SPRAY OF STOMP 30 EC @ 1 LITRE IN THE 200 LITRESOF WATER/ACRE AT PRE-EMERGENCE STAGE.',
 'THERE IS NO NEED TO IRRIGATE THE WHEAT.',
 'spray bavistin 250gm in 150lit water per acre.',
 'SUITABLE VARIETIES OF PADDY(JHONA)-PR-121,PR-122,PR-118,PR-114,PR-111,PR-116,PR-115,PR-113,PUSA-44,HKR-47,PUSA PUNJAB BASMATI-1509,PUNJAB BASMATI-3.',
 'Light rain (total 2mm), mostly falling on  night',
 'SPRAY ROGOR/METASYSTOX @ 150 ML IN 100 LITERS OF WATER/ACRE.',
 'SUITABLE VARIETIES OF PADDY(JHONA)-PR-121,PR-122,PR-118,PR-114,PR-111,PR-116,PR-115,PR-113,PUSA-44,HKR-47,PUSA PUNJAB BASMATI-1509,PUNJAB BASMATI-3.',
 'SUITABLE VARIETIES OF PADDY(JHONA)-PR-121,PR-122,PR-118,PR-114,PR-111,PR-116,PR-115,PR-113,PUSA-44,HKR-47,PUSA PUNJAB BASMATI-1509,PUNJAB BASMATI-3.',
 'SUITABLE VARIETIES OF PADDY(JHONA)-PR-121,PR-122,PR-118,PR-114,PR-111,PR-116,PR-115

In [65]:
aa = "paddy"
fin = pipeline(aa)
fin

['SUITABLE VARIETIES OF PADDY(JHONA)-PR-121,PR-122,PR-118,PR-114,PR-111,PR-116,PR-115,PR-113,PUSA-44,HKR-47,PUSA PUNJAB BASMATI-1509,PUNJAB BASMATI-3.',
 'SUITABLE VARIETIES OF PADDY(JHONA)-PR-121,PR-122,PR-118,PR-114,PR-111,PR-116,PR-115,PR-113,PUSA-44,HKR-47,PUSA PUNJAB BASMATI-1509,PUNJAB BASMATI-3.',
 'SUITABLE VARIETIES OF PADDY(JHONA)-PR-121,PR-122,PR-118,PR-114,PR-111,PR-116,PR-115,PR-113,PUSA-44,HKR-47,PUSA PUNJAB BASMATI-1509,PUNJAB BASMATI-3.',
 'SUITABLE VARIETIES OF PADDY(JHONA)-PR-121,PR-122,PR-118,PR-114,PR-111,PR-116,PR-115,PR-113,PUSA-44,HKR-47,PUSA PUNJAB BASMATI-1509,PUNJAB BASMATI-3.',
 'SUITABLE VARIETIES OF PADDY(JHONA)-PR-121,PR-122,PR-118,PR-114,PR-111,PR-116,PR-115,PR-113,PUSA-44,HKR-47,PUSA PUNJAB BASMATI-1509,PUNJAB BASMATI-3.',
 'SUITABLE VARIETIES OF PADDY(JHONA)-PR-121,PR-122,PR-118,PR-114,PR-111,PR-116,PR-115,PR-113,PUSA-44,HKR-47,PUSA PUNJAB BASMATI-1509,PUNJAB BASMATI-3.',
 'SUITABLE VARIETIES OF PADDY(JHONA)-PR-121,PR-122,PR-118,PR-114,PR-111,PR-116,PR-

## End